In [1]:
import numpy as np
import math
from random import randrange
import cPickle
import time
def unpickle(file):
    fo = open(file, 'rb')
    data = cPickle.load(fo)
    fo.close()
    return data
train_set_1 = unpickle("cifar-10-batches-py/data_batch_1")
train_set_2 = unpickle("cifar-10-batches-py/data_batch_2")
train_set_3 = unpickle("cifar-10-batches-py/data_batch_3")
train_set_4 = unpickle("cifar-10-batches-py/data_batch_4")
train_set_5 = unpickle("cifar-10-batches-py/data_batch_5")
test_set = unpickle("cifar-10-batches-py/test_batch")

In [2]:
X_train_1=train_set_1['data']
Y_train_1=train_set_1['labels']

X_test=test_set['data']
Y_test=test_set['labels']

In [3]:
print "the shape of the train_set:",X_train_1.shape
print "the shape of the test_set: ",X_test.shape

the shape of the train_set: (10000, 3072)
the shape of the test_set:  (10000, 3072)


In [3]:
print "the shape of the one image: ",X_train_1[0].shape
print "the things in the image: ",X_train_1[0]

the shape of the one image:  (3072,)
the things in the image:  [ 59  43  50 ..., 140  84  72]


Because the size of the image is 3072*1
First thing I do is reshaping the size of the image into (3,1024)
That means each patch could get (256*3)
there is a lot of ways to sperate the image
The way how do we choose the patch, no doubt it will change our resalut
1.
patch1(16*i+j) i<16 j<16 0~256
patch2(16*i+j) 16<i<32 j<16 256~512
patch3              512~768
patch4                    768~1024
2.
make a loop 
each patch get 1 pixel 
3.
....

In [4]:
def div_image(image,k):
    image=np.reshape(image,(3,1024))
    patch={}
    for i in range(k):
        patch[i]=[]
    r=1024/k
    for i in range(len(image)):
        for m in range(len(image[0])):
            index=m/r
            patch[index].append(image[i][m])
    return patch

In [5]:
def collect(data_set,k):
    patch_list=[]
    for i in range(len(data_set)):
        image=data_set[i]
        patch=div_image(image,k)
        for i in range(k):
            patch_list.append(patch[i])
    return patch_list

In [6]:
print "Start collecting the patchs of the data_batch_1...."
patch_list=collect(X_train_1,4)
patch_list=np.asarray(patch_list)
print "finish the part of collecting the patch..."

Start collecting the patchs of the data_batch_1....
finish the part of collecting the patch...


In [19]:
test_patch_list=collect(X_test,4)
test_patch_list=np.asarray(test_patch_list)

In [30]:
print "shape pf the test_patch_list ",test_patch_list.shape

shape pf the test_patch_list  (40000, 768)


In [7]:
print "the length of the patch_list ",len(patch_list)
print "the type of the patch_list ",type(patch_list[0])

the length of the patch_list  40000
the type of the patch_list  <type 'numpy.ndarray'>


In [19]:
def random_cluster_center(k,data_set):
    centroids=np.zeros(shape=(k,len(data_set[0])))
    for i in range(len(data_set[0])):
        smaller_i = min(data_set[:,i])
        largest_i = max(data_set[:,i])
        for index in range(k):
            centroids[index][i]=smaller_i + (largest_i-smaller_i)*(index+1)/float(k)
    return centroids

def dist_Eclud(variable,cluster):
    return np.sqrt(np.sum(np.power(variable-cluster,2)))

def K_means(data_set,k):
    print "Start K_means of the list of patch"
    centroids=random_cluster_center(k,data_set)
#    print "the length ",len(centroids)
    pre_labels=np.zeros(len(data_set))
#    k2label=np.zeros(k)
    convege=False
#    model_centroids=[]
    while not convege:
        convege=True
        for i in range(len(data_set)):
            image=data_set[i]
            mindis=np.inf
            label=-1
            for j in range(k):
                distance=dist_Eclud(image,centroids[j])
                if distance<mindis:
                    mindis=distance
                    label=j
            if pre_labels[i]!=label:
                convege=False
            pre_labels[i]=label
        
        for i in range(k):
            label_i_images=data_set[pre_labels==i]
            mean=label_i_images.mean(axis=0)
            centroids[i]=mean
    print "after using the K-means, we could get K cluster for the patch"
    return centroids

In [20]:
print "Start creating the dictionnary of patch....."
K=200
centroids=K_means(patch_list,K)

Start creating the dictionnary of patch.....
Start K_means of the list of patch


KeyboardInterrupt: 

After 2 hours, The K_means which is created by myself donot end
So, I try to use the K_means which is given by the package sk.learn 

In [8]:
from sklearn.cluster import KMeans 
K=200
print "Start creating the dictionary"
print "Start using the sklearn Kmeans...."
result = KMeans(n_clusters=K).fit(patch_list)
print "K-means has done..."

Start creating the dictionary
Start using the sklearn Kmeans....
K-means has done...


In [9]:
centroids=result.cluster_centers_

In [10]:
def dist_Eclud(variable,cluster):
    return np.sqrt(np.sum(np.power(variable-cluster,2)))

In [41]:
def create_dictionary(centroids,image,i,n,flag):
    patch={}
    if (flag==True):
        for m in range(n):
            patch[m]=patch_list[i*n+m]
    if flag==False:
        for m in range(n):
            patch[m]=test_patch_list[i*n+m]
    dic_patch={}
    for m in range(n):
        dic_patch[m]=np.zeros(len(centroids))
    
    d={}
    for i in range(n):
        d[i]=[]
    for centroid in centroids:
        for i in range(n):
            d[i].append(dist_Eclud(patch[i],centroid))

    for m in range(n):
        index=d[m].index(min(d[m]))
        dic_patch[m][index]=1
    if (n>=2):
        dic=np.concatenate((dic_patch[0],dic_patch[1]),axis=0)
    for i in range(2,n):
        dic=np.concatenate((dic,dic_patch[i]),axis=0)
    return dic

For each image,we create a dic.
The variable dic include,4 dic_patch
Each dic_patch shows the patch belongs to which cluster Nc

In [42]:
def image_dictionary(centroids,data_set,flag):
    image_dic=[]
    for i in range(len(data_set)):
        image=data_set[i]
        dic=create_dictionary(centroids,image,i,4,flag)
        image_dic.append(dic)
    image_dic=np.asarray(image_dic)
    return image_dic

In [43]:
train_dic=image_dictionary(centroids,X_train_1,True)
print "train dictionary has created....."

train dictionary has created.....


In [44]:
print "the shape of the image_dic",(train_dic).shape

the shape of the image_dic (10000, 800)


In [45]:
test_dic=image_dictionary(centroids,X_test,False)

In [46]:
print "the shape of the test_dic",test_dic.shape

the shape of the test_dic (10000, 800)


After creating the right input, we could choose to our own classify model or could just choose the keras to do this work.

In [47]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
print "using the OneVsRestClassifier model to build the model...."
ovrc=OneVsRestClassifier(LinearSVC(random_state=0))
model=ovrc.fit(train_dic,Y_train_1)

print "model compeleted"


using the OneVsRestClassifier model to build the model....
model compeleted


In [48]:
print model.score(test_dic,Y_test)

0.3337
